In [7]:
# %% Dump Dataset
import scipy.io
import numpy as np
import numpy.random
import numpy.matlib
import math
import cv2
import os
import pickle
import time
br = time.sleep

import tensorflow as tf
# true 3d position of the points (camera coordinate systerm)
import random
np.set_printoptions(threshold=100000)
np.set_printoptions(precision=4)
def Rand(min, max):
    return min + (max - min) * random.random()

def CheckRatation(R):
    # check the validation of ration matrix
    tR = np.matrix(R)
    ident = tR * tR.transpose()
    identErr = np.linalg.norm(ident - np.identity(3))
    assert (abs(identErr) <= 1e-10)

    det = np.linalg.det(tR)
    assert (abs(det - 1.0) <= 1e-10)

    return identErr, det

def cameraPoints(num):
    # x [-2, 2]
    # y [-2, 2]
    # z [4, 8]
    x = np.expand_dims(2. * (np.random.random_sample(num) - 0.5), axis=1)
    y = np.expand_dims(2. * (np.random.random_sample(num) - 0.5), axis=1)
    z = np.expand_dims(4. * (np.random.random_sample(num)) + 4., axis=1)
    Pc = np.concatenate([x, y, z], axis=1)
    return Pc

def project3d_2d(A, X_cam):
    #     X_cam = np.concatenate([X_cam, np.ones((np.shape(X_cam)[0], 1))], axis=1)
    X_img = np.matmul(A, np.transpose(X_cam))
    X_img[0, :] /= X_img[2, :]
    X_img[1, :] /= X_img[2, :]
    return X_img[:2,:]

def RandomRotation():
    range = 1

    # use eular formulation, three different rotation angles on 3 axis
    phi = Rand(0, range * 3.14159 * 2)
    theta = Rand(0, range * 3.14159)
    psi = Rand(0, range * 3.14159 * 2)

    R0 = []
    R0.append(math.cos(psi) * math.cos(phi) - math.cos(theta) * math.sin(phi) * math.sin(psi))
    R0.append(math.cos(psi) * math.sin(phi) + math.cos(theta) * math.cos(phi) * math.sin(psi))
    R0.append(math.sin(psi) * math.sin(theta))

    R1 = []
    R1.append(-math.sin(psi) * math.cos(phi) - math.cos(theta) * math.sin(phi) * math.cos(psi))
    R1.append(-math.sin(psi) * math.sin(phi) + math.cos(theta) * math.cos(phi) * math.cos(psi))
    R1.append(math.cos(psi) * math.sin(theta))

    R2 = []
    R2.append(math.sin(theta) * math.sin(phi))
    R2.append(-math.sin(theta) * math.cos(phi))
    R2.append(math.cos(theta))

    R = []
    R.append(R0)
    R.append(R1)
    R.append(R2)

    # print(R)
    CheckRatation(R)
    return np.array(R)

def reproj_err(x, x_err):
    # x->N*2
    # x_err->N*2
    e = x - x_err.T
    e = np.square(e)
    e = np.sqrt(np.sum(e, axis=1))
    return e

def addReprojErr(x, noise=1.e2, seed=17):
    # x: 2,n
    # 我们假定x都已经完成归一化了。
    # noise level 施加具体数值多大的噪声到数据上面。
    rng = np.random.RandomState(seed)
    mu, sigma = 0., noise/10.
    x_n = np.ones([1, np.shape(x)[1]])* (noise/2.) + \
            rng.normal(mu, sigma, [1, np.shape(x)[1]])
    
    y_n = np.ones_like(x_n) * (noise**2) - np.square(x_n)
    y_n = np.sqrt(y_n)
    
    #     x_noise = np.concatenate([x[0, :] + x_n, x[1,:] + y_n, np.ones_like(x_n)], axis=1)
    x_noise = np.concatenate([x[0, :] + x_n, x[1,:] + y_n], axis=1)
    return np.reshape(x_noise, np.shape(x))

def generateOneSample(num_3d, A, sigma, mu, num_outliers):
    Pc = cameraPoints(num_3d)
    # project points into the image plane.
    width=640
    height=480
    X_img = project3d_2d(A, Pc)
    # check if the point is within the limits of the image
    if np.max(X_img, 1)[0] > width or np.min(X_img, 1)[0] < 0\
        or np.max(X_img, 1)[1] > height or np.min(X_img, 1)[1] < 0:
        print(np.max(X_img, 1))
        print(np.min(X_img, 1))
        print("Cross the bound, bro")
    # generate additive noise
    x_noise = np.random.normal(mu, sigma, np.shape(X_img)[1])
    y_noise = np.random.normal(mu, sigma, np.shape(X_img)[1])
    add_noise = np.concatenate([np.expand_dims(x_noise, axis=0),
                                np.expand_dims(y_noise, axis=0)], axis=0)
    # adding additive noise
    X_img_noise = X_img + add_noise
    # generate random point and use it as outlier.
    rng = np.random.RandomState()
    x_out = rng.randint(1, high=width-1, size=num_outliers)
    y_out = rng.randint(1, high=height-1, size=num_outliers)
    coor_out = np.concatenate([x_out[None], y_out[None]], axis=0)
    # 2d coordinate is random, but 3D points are accurate.
    X_img_noise[:, :num_outliers] = coor_out
    
    # reproject image point to the camera coordiante system.
    Pc_noise = np.matmul(np.linalg.inv(A), 
                         np.concatenate([X_img_noise, np.ones((1,np.shape(X_img_noise)[1]))],axis=0))
    # The observed data isn't in the camera coordinate system
    # We move the center of the world system to the centroid of the 3d point cloud.
    # We also assume that the data is rotated with respect to the camera coordinate system. 
    
    t_gt = np.mean(Pc, axis=0)
    center_rep = np.matlib.repmat(t_gt, np.shape(Pc)[0],1) # n, 3
    R = RandomRotation()
    Pw = np.matmul(R, np.transpose(Pc - center_rep))
    R_gt = np.linalg.inv(R)
    # Pc: groundtruth of the 3d points in the camera coordinate system.
    x = np.concatenate([Pc_noise[:2,:], Pw], axis=0)
    x = np.transpose(x)
    idx = np.arange(np.shape(x)[0])
    np.random.shuffle(idx)
    x = x[idx]
    return x, R_gt, t_gt, X_img_noise


num_tr = 100
num_te = 100
num_va = 100

xs_tr = []
Rs_tr = []
ts_tr = []

xs_te = []
Rs_te = []
ts_te = []

xs_va = []
Rs_va = []
ts_va = []

u0 = 320
v0 = 240
fx = 800
fy = 800
A = np.array([[fx, 0., u0],[0., fy, v0],[0., 0., 1.]])
d_list = [(xs_tr, Rs_tr, ts_tr, num_tr, 5., 200, "tr"),
          (xs_te, Rs_te, ts_te, num_te, 5., 200, "te"),
          (xs_va, Rs_va, ts_va, num_va, 5., 200, "va")]
for (xs, Rs, ts, num_Samples, std_level, num_2d_3d_matches, mode) in d_list:
    for i in range(num_Samples):
        if mode == "tr":
            std_noise = std_level
            num_outlier = int(np.random.random_sample() * 150)
        if mode == "va":
#         std_noise = (i / num_2d_3d_matches) * std_level
            std_noise = std_level
            num_outlier = int((i / num_2d_3d_matches) * 150)
        if mode == "te":
            std_noise = std_level
            num_outlier = 49
        xs_samp, Rs_samp, ts_samp, _ = generateOneSample(\
                                                  num_2d_3d_matches, A, \
                                                 std_noise, 0., num_outlier)
        xs.append(np.expand_dims(xs_samp, axis=0))
        Rs.append(Rs_samp)
        ts.append(ts_samp)

dict_tr = {"xs_tr":xs_tr,
           "Rs_tr":Rs_tr,
           "ts_tr":ts_tr,
          }
dict_va = {"xs_va":xs_va,
           "Rs_va":Rs_va,
           "ts_va":ts_va,
          }
dict_te = {"xs_te":xs_te,
           "Rs_te":Rs_te,
           "ts_te":ts_te,
          }


data_folder = "./dump_data"
data_name = "standard"

for var_mode in ["tr", "va", "te"]:
    cur_data_folder = "/".join([
        data_folder, data_name,
        "outlier-{}".format(50)
    ])
    cur_folder = os.path.join(cur_data_folder, var_mode)
    if not os.path.exists(cur_folder):
        os.makedirs(cur_folder)
    print(" -- Waiting for the data_folder to be ready")
    ready_file = os.path.join(cur_folder, "ready")
    if not os.path.exists(ready_file):
        print(" -- No ready file {}".format(ready_file))
        print(" -- Saving {} data".format(var_mode))
        var_name_list = [
            "xs",
            "Rs",
            "ts",
        ]
        for var_name in var_name_list:
            cur_var_name = var_name + "_" + var_mode
            out_file_name = os.path.join(cur_folder, cur_var_name)+".pkl"
            with open(out_file_name, "wb") as ofp:
                eval("pickle.dump({}, ofp)".format(cur_var_name))
        out_file_mat = "'" + os.path.join(cur_folder, var_mode)+".mat'"
        eval("scipy.io.savemat({0}, mdict=dict_{1})".format(out_file_mat, var_mode))
        # Mark ready
        with open(ready_file, "w") as ofp:
            ofp.write("This folder is ready\n")
    else:
        print("Done!")

 -- Waiting for the data_folder to be ready
 -- No ready file ./dump_data/standard/outlier-50/tr/ready
 -- Saving tr data
 -- Waiting for the data_folder to be ready
 -- No ready file ./dump_data/standard/outlier-50/va/ready
 -- Saving va data
 -- Waiting for the data_folder to be ready
Done!
